In [2]:
import pandas as pd
import numpy as np

# Data preparation

In [5]:
df = pd.read_csv('vegemite.csv')

shuffled_data = df.sample(n=len(df)).reset_index(drop=True)

shuffled_data.to_csv('vegemite_shuffled.csv', index=False)

shuffled_data

,FFTE Feed tank level SP,FFTE Production solids SP,FFTE Steam pressure SP,TFE Out flow SP,TFE Production solids SP,TFE Vacuum pressure SP,TFE Steam pressure SP,TFE Steam temperature SP,FFTE Feed flow SP,FFTE Out steam temp SP,...,TFE Out flow PV,TFE Product out temperature,TFE Production solids PV,TFE Production solids density,TFE Steam pressure PV,TFE Steam temperature,TFE Tank level,TFE Temperature,TFE Vacuum pressure PV,Class
0,50.0,40.5,110.00,2296.30,63.0,-40.00,120.0,80.0,9500.0,40.71,...,2641.37,0.0,65.86,1.15,119.91,74.48,83.65,74.0,-76.00,0
1,50.0,40.5,128.95,2296.30,62.0,-80.00,120.0,80.0,9500.0,50.00,...,1731.52,0.0,62.24,1.17,119.91,73.20,55.53,71.0,-78.46,2
2,50.0,42.0,125.00,1771.43,76.0,-79.86,125.0,80.0,9300.0,50.00,...,2942.93,0.0,64.77,1.25,120.23,67.43,83.80,68.0,-79.85,0
3,25.0,41.5,135.00,2038.55,5.0,-50.00,2.0,80.0,18000.0,50.00,...,1532.43,0.0,0.00,0.86,1.98,74.93,72.79,75.0,2.65,1
4,50.0,42.0,115.00,2214.29,63.0,-77.30,120.0,80.0,9500.0,50.12,...,1244.49,0.0,60.21,1.24,127.92,66.02,34.48,66.0,-82.31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15232,25.0,43.0,130.00,2255.81,45.0,-36.37,120.0,80.0,9500.0,50.00,...,3641.37,0.0,72.27,1.25,120.23,66.65,39.90,76.0,-65.40,2
15233,50.0,40.5,112.00,2119.48,42.5,-51.27,120.0,80.0,9600.0,50.00,...,3705.58,0.0,62.89,1.19,119.91,73.80,81.03,83.0,-56.24,1
15234,50.0,41.5,131.80,2038.55,49.5,-57.47,120.0,80.0,9400.0,50.00,...,3628.40,0.0,68.95,0.67,119.91,89.98,81.17,90.0,-40.72,1
15235,50.0,40.5,100.00,2119.48,42.5,-36.37,120.0,80.0,9400.0,50.00,...,2020.10,0.0,63.98,1.20,119.91,67.75,50.50,69.0,-77.39,1
